In [1]:
# Install required Python packages
!pip install -U ultralytics pycocotools opencv-python-headless tqdm
!pip install --force-reinstall numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
#Configuration
import os, json, shutil, math, yaml
from pathlib import Path

COCO_ROOT = Path('/kaggle/input/coco-2017-dataset/coco2017') 
ANN_TRAIN = COCO_ROOT / 'annotations' / 'instances_train2017.json'
ANN_VAL   = COCO_ROOT / 'annotations' / 'instances_val2017.json'
IMG_TRAIN_DIR = COCO_ROOT / 'train2017'
IMG_VAL_DIR   = COCO_ROOT / 'val2017'


In [3]:
# Output working dataset in /kaggle/working
WORK_DIR = Path('/kaggle/working/coco_subset')
IMG_OUT_TRAIN = WORK_DIR / 'images' / 'train2017'
IMG_OUT_VAL   = WORK_DIR / 'images' / 'val2017'
LBL_OUT_TRAIN = WORK_DIR / 'labels' / 'train2017'
LBL_OUT_VAL   = WORK_DIR / 'labels' / 'val2017'
for p in [IMG_OUT_TRAIN, IMG_OUT_VAL, LBL_OUT_TRAIN, LBL_OUT_VAL]:
    p.mkdir(parents=True, exist_ok=True)

In [4]:
selected_classes = [
    'person',
    'car',
    'truck',
    'bus',
    'motorcycle',
    'bicycle',
    'train',
    'traffic light',
    'stop sign',
    'fire hydrant'
]

# Preprocessing

In [5]:
def read_json(p): 
    with open(p, 'r') as f: return json.load(f)

anns_train = read_json(ANN_TRAIN)
anns_val   = read_json(ANN_VAL)


In [6]:
coco_cat_name_to_id = {c['name']: c['id'] for c in anns_train['categories']}
selected_coco_ids = [coco_cat_name_to_id[name] for name in selected_classes]
cocoid_to_newid = {coco_id: i for i, coco_id in enumerate(selected_coco_ids)}


In [7]:
#Filter annotations to only objects in selected classes and keep only images that contain them
from collections import defaultdict
def filter_coco_annotations(coco_ann, selected_ids):
    images_by_id = {img['id']: img for img in coco_ann['images']}
    anns = coco_ann['annotations']
    # keep only annotations with category_id in selected_ids
    filtered_anns = [a for a in anns if a['category_id'] in selected_ids]
    # build set of image ids to keep
    image_ids = set(a['image_id'] for a in filtered_anns)
    filtered_images = [images_by_id[iid] for iid in image_ids]
    # produce new annotations dict
    newcoco = {
        'images': filtered_images,
        'annotations': filtered_anns,
        'categories': [c for c in coco_ann['categories'] if c['id'] in selected_ids]
    }
    return newcoco

train_filtered = filter_coco_annotations(anns_train, selected_coco_ids)
val_filtered   = filter_coco_annotations(anns_val, selected_coco_ids)

print("Train images kept:", len(train_filtered['images']))
print("Train annotations kept:", len(train_filtered['annotations']))
print("Val images kept:", len(val_filtered['images']))
print("Val annotations kept:", len(val_filtered['annotations']))


Train images kept: 73583
Train annotations kept: 359515
Val images kept: 3110
Val annotations kept: 15326


In [8]:
# Convert and save label (.txt) files in YOLO format (one file per image)
# YOLO format: <class_idx> <x_center> <y_center> <width> <height>  (all normalized 0..1)

from tqdm import tqdm

def save_coco_to_yolo(coco_filtered, src_img_dir, dst_img_dir, dst_lbl_dir, cocoid_to_newid):
    # group annotations by image_id
    anns_by_image = defaultdict(list)
    for a in coco_filtered['annotations']:
        anns_by_image[a['image_id']].append(a)

    for img in tqdm(coco_filtered['images'], desc=f"Processing images in {dst_img_dir.name}"):
        filename = img['file_name']
        src_path = src_img_dir / filename
        dst_path = dst_img_dir / filename

        if not src_path.exists():
            raise FileNotFoundError(f"Image file not found: {src_path}")

        # copy image
        shutil.copy2(src_path, dst_path)

        w, h = img['width'], img['height']
        label_lines = []
        for a in anns_by_image[img['id']]:
            coco_cat = a['category_id']
            new_id = cocoid_to_newid[coco_cat]
            # bbox is [x_min, y_min, width, height] in COCO (absolute px)
            x_min, y_min, bw, bh = a['bbox']
            x_center = x_min + bw/2
            y_center = y_min + bh/2
            # normalize
            x_center /= w
            y_center /= h
            bw /= w
            bh /= h
            # clamp values (safety)
            x_center = min(max(x_center, 0.0), 1.0)
            y_center = min(max(y_center, 0.0), 1.0)
            bw = min(max(bw, 0.0), 1.0)
            bh = min(max(bh, 0.0), 1.0)
            label_lines.append(f"{new_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")

        # write label file
        label_path = dst_lbl_dir / (Path(filename).stem + '.txt')
        with open(label_path, 'w') as f:
            f.write('\n'.join(label_lines))

# Run for train and val
save_coco_to_yolo(train_filtered, IMG_TRAIN_DIR, IMG_OUT_TRAIN, LBL_OUT_TRAIN, cocoid_to_newid)
save_coco_to_yolo(val_filtered,   IMG_VAL_DIR,   IMG_OUT_VAL,   LBL_OUT_VAL,   cocoid_to_newid)


Processing images in val2017: 100%|██████████| 3110/3110 [00:21<00:00, 142.36it/s]


# 

# Model Config / Training

In [9]:
import yaml
import os

# 1. DELETE the old file
yaml_path = '/kaggle/working/coco_subset/data.yaml'
if os.path.exists(yaml_path):
    os.remove(yaml_path)

In [10]:
# Create a data.yaml file to config the yolo training model
data_yaml = {
    'path': str(WORK_DIR),  # base path for images/labels relative to this
    'train': 'images/train2017',
    'val':   'images/val2017',
    'nc': len(selected_classes),
    'names': selected_classes
}

with open(WORK_DIR / 'data.yaml', 'w') as f:
    json.dump(data_yaml, f, indent=2)

print("Wrote", WORK_DIR / 'data.yaml')
print("Contents:")
print(json.dumps(data_yaml, indent=2))


Wrote /kaggle/working/coco_subset/data.yaml
Contents:
{
  "path": "/kaggle/working/coco_subset",
  "train": "images/train2017",
  "val": "images/val2017",
  "nc": 10,
  "names": [
    "person",
    "car",
    "truck",
    "bus",
    "motorcycle",
    "bicycle",
    "train",
    "traffic light",
    "stop sign",
    "fire hydrant"
  ]
}


In [11]:
# Quick checks
def count_files(p, ext):
    return sum(1 for _ in p.rglob(f'*{ext}'))

print("Train images:", count_files(IMG_OUT_TRAIN, '.jpg') + count_files(IMG_OUT_TRAIN, '.png'))
print("Train labels:", count_files(LBL_OUT_TRAIN, '.txt'))
print("Val images:", count_files(IMG_OUT_VAL, '.jpg') + count_files(IMG_OUT_VAL, '.png'))
print("Val labels:", count_files(LBL_OUT_VAL, '.txt'))

# show a sample label
sample = next(LBL_OUT_TRAIN.glob('*.txt'))
print("Sample label file:", sample)
print(sample.read_text()[:500])


Train images: 73583
Train labels: 73583
Val images: 3110
Val labels: 3110
Sample label file: /kaggle/working/coco_subset/labels/train2017/000000386803.txt
0 0.391682 0.566664 0.114094 0.121203
0 0.555176 0.473219 0.083388 0.076187
0 0.692953 0.473508 0.108541 0.083359
0 0.560635 0.358852 0.033647 0.025953
0 0.232235 0.375602 0.041882 0.039047
0 0.625400 0.356672 0.057153 0.030719
0 0.551024 0.546188 0.195553 0.131562
0 0.311718 0.378258 0.040612 0.035672
0 0.127776 0.374664 0.046706 0.029984


In [12]:
# Train using ultralytics YOLOv8
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [13]:
# choose a model: 'yolov8n.pt' (nano) is fastest ; yolov8s.pt or yolov8m.pt better accuracy.
model = YOLO('yolov8s.pt')

# training config
results = model.train(
    data=str(WORK_DIR / 'data.yaml'),
    epochs=10,            
    imgsz=640,
    batch=16,             
    name='yolo_coco_subset_run',
    device=0             
)
# After training ultralytics writes runs to /kaggle/working/runs

Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/coco_subset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_coco_subset_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pe

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       3110      15326      0.753      0.601      0.681      0.495
                person       2693      11004      0.799      0.686       0.78      0.552
                   car        535       1932      0.703       0.57       0.63      0.412
                 truck        250        415      0.584      0.413      0.505      0.351
                   bus        189        285      0.805      0.711      0.784       0.66
            motorcycle        159        371      0.738      0.598      0.683      0.436
               bicycle        149        316      0.703      0.405      0.507      0.289
                 train        157        190      0.862      0.784       0.86      0.693
         traffic light        191        637      0.674      0.396      0.471      0.248
             stop sign         69         75      0.761      0.693      0.743      0.644
          fire hydrant         86        101      0.905      0.752      0.845      0.661
Speed: 0.1ms preproce

In [14]:
# !rm -rf /kaggle/working/runs/detect/val

# Evaluate / run validation

In [15]:
best_model = 'runs/detect/yolo_coco_subset_run2/weights/best.pt' 
m = YOLO(best_model)
metrics = m.val(data=str(WORK_DIR / 'data.yaml'), imgsz=640, batch=16)
print(metrics) 

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/yolo_coco_subset_run2/weights/best.pt'

# Test on images (example)

In [ ]:
from pathlib import Path

DATA_DIR = Path("/kaggle/working/coco_subset")
IMG_OUT_VAL = DATA_DIR / "images" / "val2017"


# Run inference on an image from the validation set and save result
img_example = next(IMG_OUT_VAL.glob('*.*'))  # pick first val image
print("Example image:", img_example)
predict = m.predict(source=str(img_example), save=True, show=False, imgsz=640) 
# saved result will be in runs/predict or runs/detect by default

# Dowload model best weights

In [ ]:
from IPython.display import FileLink, FileLinks

# Adjust this path if your run name is different
model_path = "/kaggle/working/runs/detect/val11/"

# Create a download link
FileLinks(model_path)
